In [1]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from langgraph.graph import START, END, StateGraph
from typing import TypedDict, Annotated
import operator

load_dotenv()


google_api_key = os.getenv("GEMNI_API_KEY")

if not google_api_key:
    raise ValueError("GEMNI_API_KEY not found! Please set it in your .env file.")

print("API key loaded")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite", temperature=0.3, api_key=google_api_key
)


API key loaded


In [2]:
@tool
def generate_ideas(topic: str) -> str:
    """
    Generate key ideas for a blog post.
    
    Args:
        topic: The blog post topic
    """
    return f"""
    Key Ideas for '{topic}':
    1. Introduction to {topic} and its importance
    2. Core concepts and how it works
    3. Real-world applications and use cases
    4. Future trends and developments
    5. Getting started guide for beginners
    """

@tool
def draft_content(ideas: str) -> str:
    """
    Write a first draft based on ideas.
    
    Args:
        ideas: The bullet points to expand
    """
    return f"""
    # DRAFT BLOG POST
    
    ## Introduction
    Welcome to our deep dive. {ideas.splitlines()[2] if len(ideas.splitlines()) > 2 else 'Topic'} is transforming the industry.
    
    ## Main Body
    Let's explore the core concepts. It works by leveraging data and algorithms.
    Regarding real-world applications, we see it everywhere from finance to healthcare.
    
    ## Conclusion
    The future is bright. Getting started is easier than ever.
    """

@tool
def improve_writing(draft: str) -> str:
    """
    Polish and improve the writing style.
    
    Args:
        draft: The rough draft to polish
    """
    return f"""
    #  POLISHED FINAL POST 
    
    ## Introduction
    In today's rapidly evolving tech landscape, understanding this topic is crucial. It is fundamentally reshaping our approach to problem-solving.
    
    ## Core Analysis
    At its heart, this technology leverages sophisticated algorithms to drive insights. From predictive analytics in finance to diagnostic tools in healthcare, the applications are limitless.
    
    ## Future Outlook
    As we look ahead, the barriers to entry are lowering. Now is the perfect time to start your journey.
    """

print("Pipeline tools created")

Pipeline tools created


In [3]:

class ContentState(TypedDict):
    messages: Annotated[list, operator.add]
    topic: str
    ideas: str
    draft: str
    final_post: str

# --- AGENT 1: BRAINSTORMER ---
brainstormer_prompt = SystemMessage(content="You are a creative brainstormer. Use generate_ideas tool to outline key points.")
brainstormer_llm = llm.bind_tools([generate_ideas])

def brainstormer_agent(state: ContentState) -> dict:
    topic = state["topic"]
    messages = [brainstormer_prompt, HumanMessage(content=f"Brainstorm ideas for: {topic}")]
    
    response = brainstormer_llm.invoke(messages)
    
    ideas_content = ""
    # Check for tool calls
    if response.tool_calls:
        print(" Brainstormer: Generating ideas...")
        tool_call = response.tool_calls[0]
        ideas_content = generate_ideas.invoke(tool_call["args"])
    else:
        ideas_content = response.content
        
    return {
        "messages": [AIMessage(content=f"Ideas: {ideas_content}")],
        "ideas": ideas_content
    }

# --- AGENT 2: WRITER ---
writer_prompt = SystemMessage(content="You are a skilled content writer. Use draft_content tool to write a blog post based on provided ideas.")
writer_llm = llm.bind_tools([draft_content])

def writer_agent(state: ContentState) -> dict:
    ideas = state["ideas"]
    print(" Writer: Drafting content...")
    draft = draft_content.invoke({"ideas": ideas})
    
    return {
        "messages": [AIMessage(content=f"Draft: {draft}")],
        "draft": draft
    }

# --- AGENT 3: EDITOR ---
editor_prompt = SystemMessage(content="You are a strict editor. Use improve_writing tool to polish drafts.")
editor_llm = llm.bind_tools([improve_writing])

def editor_agent(state: ContentState) -> dict:
    draft = state["draft"]
    print("📝 Editor: Polishing final post...")
    final_version = improve_writing.invoke({"draft": draft})
    
    return {
        "messages": [AIMessage(content=f"Final: {final_version}")],
        "final_post": final_version
    }

print("Agents created")

Agents created


In [6]:

pipeline_builder = StateGraph(ContentState)
pipeline_builder.add_node("brainstormer", brainstormer_agent)
pipeline_builder.add_node("writer", writer_agent)
pipeline_builder.add_node("editor", editor_agent)

pipeline_builder.add_edge(START, "brainstormer")
pipeline_builder.add_edge("brainstormer", "writer")
pipeline_builder.add_edge("writer", "editor")
pipeline_builder.add_edge("editor", END)
content_pipeline = pipeline_builder.compile()

print(" Pipeline graph compiled")

 Pipeline graph compiled


In [7]:

print(f"\n{'='*20} STARTING PIPELINE {'='*20}\n")

result = content_pipeline.invoke({
    "topic": "Artificial intelligence",
    "messages": []
})

print(f"\n{'='*20} FINAL OUTPUT {'='*20}")
print(result["final_post"])


==================== STARTING PIPELINE ====================

 Brainstormer: Generating ideas...
 Writer: Drafting content...
📝 Editor: Polishing final post...

==================== FINAL OUTPUT ====================

    #  POLISHED FINAL POST 

    ## Introduction
    In today's rapidly evolving tech landscape, understanding this topic is crucial. It is fundamentally reshaping our approach to problem-solving.

    ## Core Analysis
    At its heart, this technology leverages sophisticated algorithms to drive insights. From predictive analytics in finance to diagnostic tools in healthcare, the applications are limitless.

    ## Future Outlook
    As we look ahead, the barriers to entry are lowering. Now is the perfect time to start your journey.
    
